# Accessing the NHTSA API (crawler)
## Recall Data 

In [2]:
import requests
from pathlib import Path
import json
import pandas as pd
import urllib.request
import urllib.error
import time
from loguru import logger
import pygame
from IPython.display import display, clear_output

In [12]:
# Set API prefix because our api headers are same.
api_prefix = "https://api.nhtsa.gov"

# Set API suffix for get year.
api_get_year = "/products/vehicle/modelYears?issueType=r"

# Set output path.
dir_raw = Path("../raw/Recalls")
dir_raw.mkdir(parents=True, exist_ok=True)
out_dir = Path("../raw/Recalls_buffer")
out_dir.mkdir(parents=True, exist_ok=True)

In [10]:
def fetch_recalls(url):
    logger.info("Fetching %s" % url)
    try:
        headers = {
          'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
        }
        response = requests.get(url,headers=headers)
        status = response.status_code
        logger.info(status)

        # Stop if the API cannot be accessed properly.
        if status != 200:
            return [-1,None]
        
        # Otherwise return retrieved data.
        raw_data = response.text
        return [1,json.loads(raw_data)]
    except urllib.error.URLError as e:
        e.reason 

# Output each recall for a json file.
def write_out_raw(recalls_count,year,make,model,data):
    # Check if '/' is already in make or model, replaced by '_or_'.
    if '/' in make:
        make = make.replace("/", "_or_")
    if '/' in model:
        model = model.replace("/", "_or_")
    # File name format [complaint_num]year_make_model.json
    #             e.g. [802]1994_MERCEDES BENZ_E.json
    fname = "[%d]%s_%s_%s.json" % (complaints_count,year,make,model)
    out_path = dir_raw / fname
    logger.info("Writing data to %s" % out_path)
    fout = open(out_path, "w")
    json.dump(data, fout, indent=4, sort_keys=True)
    fout.close()
    
# Check API status.
def check_status(data):
    #number of recall in this time.
    recall_num = data["count"]
    # Are results returned successfully?
    if data["message"] != "Results returned successfully":
        logger.warning("Fail to get from API, %s" % (data["status_message"]))
        return [False,-1]
    elif recall_num <= 0:
        logger.warning("No info from current API address.")
        return [False,0]
    else:
        return [True,recall_num]

# Output for "years", "makes", "models" which are dictionaries we get from first 3 steps.  
def write_out(name,data):
    fname = "%s.json" % (name)
    out_path = out_dir / fname
    logger.info("Writing data to %s" % out_path)
    fout = open(out_path, "w")
    json.dump(data, fout, indent=4, sort_keys=True)
    fout.close()

## Step-1: Get all Model Years 

In [11]:
# Construct API url.
api = api_prefix + api_get_year
# Fetch data.
status,data = fetch_recalls(api)
# Convert result data into list format.
years = (list(each["modelYear"] for each in data["results"]))
logger.info("Completed the task of getting all model years!")
# Store years data.
write_out("years",years)

2022-04-06 23:53:20.869 | INFO     | __main__:fetch_recalls:2 - Fetching https://api.nhtsa.gov/products/vehicle/modelYears?issueType=r
2022-04-06 23:53:21.227 | INFO     | __main__:fetch_recalls:9 - 200
2022-04-06 23:53:21.227 | INFO     | __main__:<module>:7 - Completed the task of getting all model years!
2022-04-06 23:53:21.228 | INFO     | __main__:write_out:55 - Writing data to ../raw/Recalls_buffer/years.json


## Step-2: Get all Makes for the Model Year

In [13]:
makes={}
for year in years:
    clear_output()
    # Construct API url.
    api = api_prefix + "/products/vehicle/makes?modelYear=%d&issueType=r" % (int(year))
    # Fetch data.
    status,data = fetch_recalls(api)
    # Convert result data into list format then store in a dict by year.
    makes[year] = (list(each["make"] for each in data["results"]))
logger.info("Completed the task of getting all makes for the model year!")
# Store makes data.
write_out("recall_makes",makes)

2022-04-06 23:55:07.139 | INFO     | __main__:fetch_recalls:2 - Fetching https://api.nhtsa.gov/products/vehicle/makes?modelYear=9999&issueType=r
2022-04-06 23:55:08.055 | INFO     | __main__:fetch_recalls:9 - 200
2022-04-06 23:55:08.056 | INFO     | __main__:<module>:10 - Completed the task of getting all makes for the model year!
2022-04-06 23:55:08.057 | INFO     | __main__:write_out:55 - Writing data to ../raw/Recalls_buffer/recall_makes.json


## Step-3: Get all Models for the Make and Model Year 
Due to the website's restrictions on crawlers, we have to change IP frequently to bypass its restrictions. The following function will automatically stop when it received an access error (Typically a 403 error) and return the data we got so far. And it will give us a hint for next start.

In [53]:
def Get_model(years,skip):
    models={}
    start_year = years[0]
    for year in years:
        for make in makes[year]:
            # If meet some special case we need skip it.
            if make != skip:
                clear_output()
                api = api_prefix + "/products/vehicle/models?modelYear=%d&make=%s&issueType=r" % (int(year),make)
                status,data = fetch_recalls(api)
            # Status equal -1 means we are failing to use this API at this moment.
            if status == -1:
                logger.warning("Start year:%s, Current Year:%s(index:%d) Current Make:%s(index:%d)" % (start_year,year,years.index(year),make,makes[year].index(make)))
                return models
            # Otherwise, convert result data into list format then store in a dict by make.
            else:
                models[make] = (list(each["model"] for each in data["results"]))
    end_year = year
    logger.info("Finished (%s-%s)" % (start_year,end_year))
    return models

# Maybe:!!!!!

#### We cannot access url related to **"EMERGENCY ONE"**, therefore we skip it.

In [54]:
models = Get_model(years,"EMERGENCY ONE")

2022-04-05 23:43:22.630 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/products/vehicle/models?modelYear=2011&make=NU WA&issueType=c
2022-04-05 23:43:22.809 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-05 23:43:22.810 | WARNING  | __main__:Get_model:13 - Start year:1949, Current Year:2011(index:56) Current Make:NU WA(index:102)


In [57]:
start = years.index("2011")
model_2 = Get_model(years[start:],"EMERGENCY ONE")

2022-04-05 23:51:34.639 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/products/vehicle/models?modelYear=9999&make=ZAP&issueType=c
2022-04-05 23:51:34.837 | INFO     | __main__:fetch_complaints:9 - 200
2022-04-05 23:51:34.838 | INFO     | __main__:Get_model:18 - Finished (2011-9999)


In [63]:
len(models),len(model_2)

(709, 517)

#### Define a function for merge two dictionary with out replacement & duplication.

In [60]:
def merge_2_dict(dict_1,dict_2):
    for key, value in dict_2.items():
        if key in dict_1:
            for v in value:
                if v not in dict_1[key]:
                    dict_1[key].append(v)
        else:
            dict_1[key] = value
    return dict_1

In [70]:
models = merge_2_dict(models,model_2)

#### Delete pair if the value of this key is empty.

In [71]:
len(models)

934

In [72]:
clean_models = models.copy()
for key in list(clean_models.keys()):
    if not clean_models.get(key):
        del clean_models[key]
len(clean_models)

860

#### Delect special 403 case from **makes** and **models**.

In [228]:
#Retrieve back from file

#f = open('../raw/Complaints-Plus/makes.json')
#data_m = json.load(f)
#f.close()
#makes = data_m.copy()

In [229]:
makes_clean_403 = makes.copy()
for k,v in makes_clean_403.items():
    if "EMERGENCY ONE" in v:
        id = v.index("EMERGENCY ONE")
        del v[id]
logger.info( "%d -> %d" %(len(makes),len(makes_clean_403)))

2022-04-06 15:14:27.720 | INFO     | __main__:<module>:6 - 69 -> 69


In [230]:
models_clean_403 = models.copy()
del models_clean_403["EMERGENCY ONE"]
logger.info( "%d -> %d" %(len(models),len(models_clean_403)))

2022-04-06 15:14:32.666 | INFO     | __main__:<module>:3 - 934 -> 933


In [77]:
write_out("recall_models",models)

2022-04-06 00:04:51.999 | INFO     | __main__:write_out:44 - Writing data to ../raw/Complaints-Plus/models.json
2022-04-06 00:04:52.006 | INFO     | __main__:write_out:44 - Writing data to ../raw/Complaints-Plus/makes.json
2022-04-06 00:04:52.013 | INFO     | __main__:write_out:44 - Writing data to ../raw/Complaints-Plus/years.json


## Step-4: Get all recalls for the selected Model Year, Make, Model

In [146]:
def Get_recalls(recalls_count,years,makes,models,continue_):
    recalls_count = recalls_count
    start_year = years[0]
    for year in years:
        # "continue_" means we've suffered an interruption, 
        # and we're going back to that position.
        if continue_ : 
            #Get user input of "make" which want to continue.
            make_input = input()
            
            #Find correspond index position in the "makes" dict.
            position_make = makes[year].index(make_input)
            
            #Get user input of "model" which want to continue.
            model_input = input()
            
            #Find correspond index position in the "models" dict.
            position_model = models[make_input].index(model_input)
            
            # Find a list of rest of makes by current year. 
            make_position = makes[year].index(make_input)
            
            # Find a list of rest of models by current year & make. 
            model_position = models[make_input].index(model_input)

            for make in (makes[year])[make_position:]:
                for model in (models[make_input])[model_position:]:
                    
                    clear_output()
                    api = api_prefix + "/recalls/recallsByVehicle?make=%s&model=%s&modelYear=%d" % (make,model,int(year))
                    # Fetch data.
                    status, data = fetch_recalls(api)

                    # Status equal -1 means we are failing to use this API at this moment.
                    if status == -1:
                        logger.warning("Stopped! (%s_%s_%s_%s)" % (recalls_count,year,make,model))                    
                        alarm_clock()
                        return -1
                    
                    # Skip if data is empty
                    if data["results"] != []:
                        #Write out
                        write_out_raw(recalls_count,year,make,model,data["results"])
                        recalls_count += 1
                    else:
                        logger.info("Empty data")
            continue_ = False
        else:
            for make in makes[year]:
                for model in models[make]:
                    clear_output()
                    api = api_prefix + "/recalls/recallsByVehicle?make=%s&model=%s&modelYear=%d" % (make,model,int(year))
                    status, data = fetch_recalls(api)

                    # Status equal -1 means we are failing to use this API at this moment.
                    if status == -1:
                        logger.warning("Stopped! (%s_%s_%s_%s)" % (recalls_count,year,make,model)) 
                        alarm_clock()
                        return -1
                    
                    # Skip if data is empty
                    if data["results"] != []:
                        #Write out
                        write_out_raw(recalls_count,year,make,model,data["results"])
                        recalls_count += 1
                    else:
                        logger.info("Empty data")
    end_year = year
    logger.info("Finished (%s_%s_%s_%s) from %s to %s." % (recalls_count,year,make,model,start_year,end_year))

# Set an alert to remind me to reset when our IP is blocked.
def alarm_clock():
    file = 'alarm_music.mp3'
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(file)
    pygame.mixer.music.play()
    time.sleep(10)
    pygame.mixer.music.stop()

In [97]:
Get_recalls(0,years,makes,models,False)

2022-04-06 00:27:28.693 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=CADILLAC&model=STS&modelYear=1972
2022-04-06 00:27:28.898 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 00:27:28.900 | WARNING  | __main__:Get_complaints:36 - Stopped! (27_1972_CADILLAC_STS)


-1

In [135]:
start = years.index("1972")
Get_recalls(20,years[start:],makes,models,True)

2022-04-06 12:27:26.238 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=BMW&model=550I&modelYear=1978
2022-04-06 12:27:26.286 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 12:27:26.288 | WARNING  | __main__:Get_complaints:53 - Stopped! (49_1978_BMW_550I)


-1

In [141]:
start = years.index("1978")
Get_recalls(49,years[start:],makes,models,True)

2022-04-06 13:32:34.714 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=MERCEDES BENZ&model=ML350&modelYear=1979
2022-04-06 13:32:34.895 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 13:32:34.896 | WARNING  | __main__:Get_complaints:56 - Stopped! (63_1979_MERCEDES BENZ_ML350)


-1

In [143]:
start = years.index("1979")
Get_recalls(63,years[start:],makes,models,True)

2022-04-06 14:06:41.186 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=FORD&model=FLEX&modelYear=1986
2022-04-06 14:06:41.381 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 14:06:41.382 | WARNING  | __main__:Get_complaints:56 - Stopped! (236_1986_FORD_FLEX)


-1

In [ ]:
start = years.index("1986")
Get_recalls(236,years[start:],makes,models,True)

2022-04-06 15:45:16.631 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=FORD&model=TAURUS&modelYear=1992


In [149]:
start = years.index("19")
Get_recalls(311,years[start:],makes,models,True)

2022-04-06 14:23:33.468 | INFO     | __main__:fetch_complaints:2 - Fetching https://api.nhtsa.gov/complaints/complaintsByVehicle?make=EMERGENCY ONE&model=ATTITUDE&modelYear=1988
2022-04-06 14:23:34.529 | INFO     | __main__:fetch_complaints:9 - 403
2022-04-06 14:23:34.530 | WARNING  | __main__:Get_complaints:36 - Stopped! (311_1988_EMERGENCY ONE_ATTITUDE)


-1